# dlt & SQLMesh Runner

## Set Environment Vars

### Load From KeyVault

In [54]:
import os

credentials = [
    "CREDENTIALS__AZURE_TENANT_ID",
    "CREDENTIALS__AZURE_CLIENT_ID",
    "CREDENTIALS__AZURE_CLIENT_SECRET",
]

for credential in credentials:
    secret = credential.replace("_", "-")
    value = notebookutils.credentials.getSecret("https://mattiasthalen-fabric.vault.azure.net/", secret)
    os.environ[credential] = value

    print(f"{credential} = {os.getenv(credential)}")

CREDENTIALS__AZURE_TENANT_ID = [REDACTED]
CREDENTIALS__AZURE_CLIENT_ID = [REDACTED]
CREDENTIALS__AZURE_CLIENT_SECRET = [REDACTED]


### Load From Variable Library

In [55]:
project_vars = notebookutils.variableLibrary.getLibrary("project_vars")
vars = [
    "FABRIC__WORKSPACE_ID",
    "FABRIC__WAREHOUSE_ENDPOINT",
    "FABRIC__DATABASE_ENDPOINT",
    "FABRIC__DATABASE",
    "DESTINATION__BUCKET_URL",
]

for var in vars:
    value = project_vars[var]
    os.environ[var] = value
    print(f"{var} = {os.getenv(credential)}")


FABRIC__WORKSPACE_ID = [REDACTED]
FABRIC__WAREHOUSE_ENDPOINT = [REDACTED]
FABRIC__DATABASE_ENDPOINT = [REDACTED]
FABRIC__DATABASE = [REDACTED]
DESTINATION__BUCKET_URL = [REDACTED]


## Clone Project

In [56]:
import git

organization = "mattiasthalen"
repo_name = "northwind-to-fabric"

url = f"https://github.com/{organization}/{repo_name}.git"

branch = "main"
code_path = f"/lakehouse/default/Files/{repo_name}"

try:
    repo = git.Repo.clone_from(
        url=url,
        to_path=code_path,
        branch=branch,
        depth=1,
        single_branch=True
    )
except git.GitCommandError as e:
    if e.status == 128:
        repo = git.Repo(code_path)
        origin = repo.remotes.origin
        origin.pull(branch)
    else:
        raise

## Install Dependencies

In [57]:
%pip install "dlt[az,deltalake,filesystem,parquet]>=1.12.1"
%pip install "enlighten>=1.14.1"
%pip install "ibis==3.3.0"
%pip install "networkx>=3.5"
%pip install "sqlmesh[fabric,mssql,lsp]"

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


## Run Project

### Run dlt

In [66]:
import subprocess

cmd = ["python", "dlt/northwind.py", "prod"]
result = subprocess.run(
    cmd,
    cwd=code_path,
    text=True,
    check=True
) 

Running in prod mode
Pipeline northwind load step completed in 5.99 seconds
1 load package(s) were loaded to destination filesystem and into dataset northwind
The filesystem destination used abfss://99a969c7-6acc-45c8-808f-93eea0b7b50e@onelake.dfs.fabric.microsoft.com/2c98cc5e-bff5-4fcb-b459-d698c8d4dea9/Tables location to store data
Load package 1755698255.9022968 is LOADED and contains no failed jobs


### Run SQLMesh

In [ ]:
import subprocess

cmd = ["sqlmesh", "plan", "prod", "--run", "--auto-apply", "--no-prompts"]
result = subprocess.run(
    cmd,
    cwd=code_path,
    text=True,
    check=True
) 